### SlideRule API

SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

SlideRule can be accessed by any http client (e.g. curl) by making GET and POST requests to the SlideRule service. For the purposes of this document, all requests to SlideRule will originate from a Python script using Python's requests module.

This notebook uses [Jupyter widgets](https://ipywidgets.readthedocs.io) to set parameters for using the SlideRule API.
Regions of interest for submitting to SlideRule are drawn on a [leaflet](https://ipyleaflet.readthedocs.io) map.  Multiple polygons can be submitted at a given time.

#### Load necessary packages

In [ ]:
from sliderule import icesat2, ipysliderule, io
import time
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import ipywidgets as widgets
# autoreload
%load_ext autoreload
%autoreload 2

#### Set options for making science data processing requests to SlideRule

In [ ]:
# set the url for the sliderule service.
icesat2.init("icesat2sliderule.org", False)

# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
widgets.VBox([
    SRwidgets.asset,
    SRwidgets.release,
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
    SRwidgets.projection
])

#### Select regions of interest for submitting to SlideRule

In [ ]:
# create ipyleaflet map in projection
m = ipysliderule.leaflet(SRwidgets.projection.value)
m.map

#### Build and transmit requests to SlideRule

In [ ]:
# sliderule asset and data release
asset = SRwidgets.asset.value
release = SRwidgets.release.value
# get sliderule submission time
submission_time = time.strftime('%Y%m%d%H%M%S',time.gmtime())

# create an empty dataframe
gdf = icesat2.__emptyframe()
# for each region of interest
for poly in m.regions:
    # build sliderule parameters
    parms = {
        # polygon from map
        "poly": poly,
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": SRwidgets.surface_type.index,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # maximum iterations, not including initial least-squares-fit selection
        "maxi": SRwidgets.iteration.value,
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        # minimum height of PE window in meters
        "H_min_win": SRwidgets.window.value,
        # maximum robust dispersion in meters
        "sigma_r_max": SRwidgets.sigma.value
    }
    # make the request to the SlideRule (ATL06-SR) endpoint
    # and pass it the request parameters to request ATL06 Data
    gdf = gdf.append(icesat2.atl06p(parms, asset, version=release))

#### Create output plots

1) Location globally of ground tracks output from SlideRule
2) Local map of segment elevations output from SlideRule

In [ ]:
# output map of resource location
f1, ax1 = plt.subplots(num=1, nrows=1, ncols=1, figsize=(10,6),
    subplot_kw=dict(projection=cartopy.crs.PlateCarree()))
ax1.plot(gdf.geometry.values.x,gdf.geometry.values.y,'r.',
    ms=1,transform=cartopy.crs.Geodetic())
ax1.set_extent((-180,180,-90,90),crs=cartopy.crs.PlateCarree())
# add coastlines with filled land and lakes
ax1.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
ax1.add_feature(cartopy.feature.LAKES)
ax1.gridlines(xlocs=np.arange(-180.,181.,60.), ylocs=np.arange(-90.,91.,30.))
# add title
ax1.set_title('Ground Tracks')
# stronger linewidth on frame
ax1.spines['geo'].set_linewidth(2.0)
ax1.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
f1.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)

# output map of heights
f2, ax2 = plt.subplots(num=2, nrows=1, ncols=1, figsize=(10,6),
    subplot_kw=dict(projection=cartopy.crs.PlateCarree()))
# create scatter plot of elevations
sc = ax2.scatter(gdf.geometry.values.x,gdf.geometry.values.y,
    c=gdf.h_mean,s=1,edgecolor='none',cmap=plt.cm.viridis,
    transform=cartopy.crs.PlateCarree())
# extract latitude and longitude of polygon
for poly in m.regions:
    lon,lat = ipysliderule.coordinates(poly)
    ax2.plot(lon, lat, 'r', lw=1.5, transform=cartopy.crs.PlateCarree())
# add coastlines with filled land and lakes
ax2.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
ax2.add_feature(cartopy.feature.LAKES)
#-- Add colorbar axes at position rect [left, bottom, width, height]
cbar_ax = f2.add_axes([0.87, 0.015, 0.0325, 0.94])
#-- add extension triangles to upper and lower bounds
cbar = f2.colorbar(sc, cax=cbar_ax, extend='both', extendfrac=0.0375,
    drawedges=False, orientation='vertical')
#-- rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
#-- Add label to the colorbar and adjust coordinates
cbar.ax.set_ylabel('Height above WGS84 Ellipsoid',labelpad=10)
cbar.ax.set_xlabel('m', rotation=0)
cbar.ax.xaxis.set_label_coords(0.5, 1.05)
cbar.ax.tick_params(which='both', width=1, direction='in')
# stronger linewidth on frame
ax2.spines['geo'].set_linewidth(2.0)
ax2.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
f2.subplots_adjust(left=0.02,right=0.86,bottom=0.05,top=0.98)
# show the figures
plt.show()

#### Save GeoDataFrame to HDF5
Outputs as a [pytables](https://www.pytables.org/) HDF5 which is easily read back as a Geopandas GeoDataFrame

In [ ]:
args = (submission_time,release)
io.to_file(gdf,"ATL06-SR_{0}_{1}.h5".format(*args),
    format='hdf', parameters=parms, regions=m.regions, verbose=True)

#### Read GeoDataFrame from HDF5
Read the HDF5 back as a Geopandas GeoDataFrame for later use

In [ ]:
args = (submission_time,release)
gdf = io.from_file("ATL06-SR_{0}_{1}.h5".format(*args),format='hdf')